In [1]:
import joblib
# ============================================================================
# TELCO CUSTOMER CHURN PREDICTION - ANN WITH HYPERPARAMETER TUNING
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, confusion_matrix, classification_report,
                           roc_auc_score, roc_curve)
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# TensorFlow/Keras for ANN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.regularizers import l1, l2, l1_l2

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# ============================================================================
# 1. DATA LOADING AND INITIAL EXPLORATION
# ============================================================================

# Load the dataset (adjust path as needed)
# Dataset available at: https://www.kaggle.com/datasets/blastchar/telco-customer-churn
try:
    url = "https://raw.githubusercontent.com/Chanura04/ML-Coursework/main/dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv"

    df = pd.read_csv(url)
    print("Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"\nFirst 5 rows:")
    print(df.head())
except FileNotFoundError:
    print("Dataset file not found. Please download from Kaggle and update the path.")
    # Create sample data structure for reference
    df = pd.DataFrame()

# Display basic information
print("\n" + "="*50)
print("DATASET INFORMATION")
print("="*50)
print(df.info())

print("\n" + "="*50)
print("DATASET STATISTICS")
print("="*50)
print(df.describe())

# Check for missing values
print("\n" + "="*50)
print("MISSING VALUES")
print("="*50)
print(df.isnull().sum())

# ============================================================================
# 2. DATA PREPROCESSING
# ============================================================================

def preprocess_data(df):
    """
    Preprocess the Telco Customer Churn dataset
    """
    # Create a copy to avoid modifying original
    data = df.copy()

    # Handle TotalCharges column - convert to numeric, errors='coerce' will set invalid to NaN
    data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors='coerce')

    # Check class distribution
    print("\nClass distribution (Churn):")
    print(data['Churn'].value_counts())
    print(f"\nChurn rate: {(data['Churn'].value_counts()[1] / len(data)) * 100:.2f}%")

    # Drop customerID as it's not useful for prediction
    if 'customerID' in data.columns:
        data = data.drop('customerID', axis=1)

    # Handle missing values
    print(f"\nMissing values after conversion: {data.isnull().sum().sum()}")

    # Impute missing values in TotalCharges with median
    if data['TotalCharges'].isnull().sum() > 0:
        median_total_charges = data['TotalCharges'].median()
        data['TotalCharges'] = data['TotalCharges'].fillna(median_total_charges)
        print(f"Imputed {data['TotalCharges'].isnull().sum()} missing values in TotalCharges")

    # Separate features and target
    X = data.drop('Churn', axis=1)
    y = data['Churn']

    # Encode target variable
    y = y.map({'Yes': 1, 'No': 0})

    # Identify categorical and numerical columns
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    print(f"\nCategorical columns: {categorical_cols}")
    print(f"Numerical columns: {numerical_cols}")

    # Handle specific binary categorical columns
    binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService',
                   'PaperlessBilling', 'SeniorCitizen']  # SeniorCitizen is already 0/1

    # For binary columns, use Label Encoding
    for col in binary_cols:
        if col in categorical_cols:
            if col == 'gender':
                X[col] = X[col].map({'Female': 0, 'Male': 1})
            elif col in ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']:
                X[col] = X[col].map({'No': 0, 'Yes': 1})

    # Update categorical columns list after encoding binary columns
    categorical_cols = [col for col in categorical_cols if col not in binary_cols]

    # Create preprocessing pipelines
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # Combine preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    return X, y, preprocessor, categorical_cols, numerical_cols

# Preprocess the data
if not df.empty:
    X, y, preprocessor, categorical_cols, numerical_cols = preprocess_data(df)

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    print(f"\nTraining set shape: {X_train.shape}")
    print(f"Test set shape: {X_test.shape}")

    # Apply preprocessing
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    # Get feature names after one-hot encoding
    feature_names = numerical_cols.copy()
    if categorical_cols:
        ohe = preprocessor.named_transformers_['cat'].named_steps['onehot']
        categorical_features = ohe.get_feature_names_out(categorical_cols)
        feature_names.extend(categorical_features)

    print(f"\nProcessed feature shape: {X_train_processed.shape}")
    print(f"Number of features: {len(feature_names)}")

# ============================================================================
# 3. ANN MODEL BUILDING FUNCTION
# ============================================================================

def create_ann_model(learning_rate=0.001,
                     hidden_layers=2,
                     neurons_per_layer=64,
                     dropout_rate=0.3,
                     activation='relu',
                     optimizer='adam',
                     l1_reg=0.001,
                     l2_reg=0.001):
    """
    Create a configurable ANN model for binary classification
    """
    model = Sequential()

    # Input layer
    model.add(Dense(neurons_per_layer,
                    input_dim=X_train_processed.shape[1],
                    activation=activation,
                    kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    # Hidden layers
    for i in range(hidden_layers - 1):
        model.add(Dense(neurons_per_layer,
                        activation=activation,
                        kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))

    # Output layer (binary classification)
    model.add(Dense(1, activation='sigmoid'))

    # Compile model
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        opt = Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=opt,
        loss='binary_crossentropy',
        metrics=['accuracy',
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 tf.keras.metrics.AUC(name='auc')]
    )

    return model

# ============================================================================
# 4. HYPERPARAMETER TUNING USING GRID SEARCH
# ============================================================================

def perform_hyperparameter_tuning(X_train, y_train, X_test, y_test):
    """
    Perform hyperparameter tuning for the ANN model
    """
    print("\n" + "="*50)
    print("HYPERPARAMETER TUNING - ANN MODEL")
    print("="*50)

    # Create KerasClassifier wrapper for scikit-learn compatibility
    ann_model = KerasClassifier(
        build_fn=create_ann_model,
        verbose=0
    )

    # Define hyperparameter grid
    param_grid = {
        'batch_size': [32, 64, 128],
        'epochs': [50, 100],
        'learning_rate': [0.001, 0.01, 0.0001],
        'hidden_layers': [1, 2, 3],
        'neurons_per_layer': [32, 64, 128],
        'dropout_rate': [0.2, 0.3, 0.5],
        'activation': ['relu', 'tanh'],
        'optimizer': ['adam', 'rmsprop'],
        'l1_reg': [0.0, 0.001, 0.01],
        'l2_reg': [0.0, 0.001, 0.01]
    }

    # Note: Full grid search can be computationally expensive
    # For demonstration, we'll use a smaller subset or RandomizedSearchCV

    # Alternatively, use RandomizedSearchCV for faster tuning
    from sklearn.model_selection import RandomizedSearchCV

    # Create a smaller parameter distribution for randomized search
    param_dist = {
        'batch_size': [32, 64],
        'epochs': [50, 80],
        'learning_rate': [0.001, 0.01],
        'hidden_layers': [2, 3],
        'neurons_per_layer': [64, 128],
        'dropout_rate': [0.3, 0.5],
        'activation': ['relu', 'tanh'],
        'optimizer': ['adam'],
        'l1_reg': [0.0, 0.001],
        'l2_reg': [0.001, 0.01]
    }

    # Create callbacks for early stopping
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=0
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=0
    )

    # Manual hyperparameter tuning approach (more practical)
    print("\nManual Hyperparameter Tuning Results:")
    print("-" * 40)

    # Test different configurations manually
    configurations = [
        {
            'name': 'Config 1 - Basic',
            'params': {
                'learning_rate': 0.001,
                'hidden_layers': 2,
                'neurons_per_layer': 64,
                'dropout_rate': 0.3,
                'activation': 'relu',
                'optimizer': 'adam',
                'batch_size': 32,
                'epochs': 100
            }
        },
        {
            'name': 'Config 2 - Deeper',
            'params': {
                'learning_rate': 0.001,
                'hidden_layers': 3,
                'neurons_per_layer': 128,
                'dropout_rate': 0.5,
                'activation': 'relu',
                'optimizer': 'adam',
                'batch_size': 64,
                'epochs': 100
            }
        },
        {
            'name': 'Config 3 - Regularized',
            'params': {
                'learning_rate': 0.0001,
                'hidden_layers': 2,
                'neurons_per_layer': 64,
                'dropout_rate': 0.3,
                'activation': 'tanh',
                'optimizer': 'adam',
                'batch_size': 32,
                'epochs': 150,
                'l1_reg': 0.001,
                'l2_reg': 0.001
            }
        }
    ]

    best_model = None
    best_score = 0
    best_config = {}
    results = []

    for config in configurations:
        print(f"\nTesting {config['name']}...")

        # Create and train model
        model = create_ann_model(**{k: v for k, v in config['params'].items()
                                    if k in ['learning_rate', 'hidden_layers',
                                            'neurons_per_layer', 'dropout_rate',
                                            'activation', 'optimizer', 'l1_reg', 'l2_reg']})

        history = model.fit(
            X_train_processed, y_train,
            validation_split=0.2,
            batch_size=config['params']['batch_size'],
            epochs=config['params']['epochs'],
            callbacks=[early_stopping, reduce_lr],
            verbose=0
        )

        # Evaluate on test set
        y_pred_proba = model.predict(X_test_processed, verbose=0)
        y_pred = (y_pred_proba > 0.5).astype(int)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred_proba)

        results.append({
            'config_name': config['name'],
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc': auc,
            'params': config['params']
        })

        print(f"  Accuracy: {accuracy:.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1:.4f}")
        print(f"  AUC-ROC: {auc:.4f}")

        # Update best model
        if f1 > best_score:  # Using F1-score as selection criteria
            best_score = f1
            best_model = model
            best_config = config

    # Display results summary
    print("\n" + "="*50)
    print("HYPERPARAMETER TUNING SUMMARY")
    print("="*50)

    results_df = pd.DataFrame(results)
    print(results_df[['config_name', 'accuracy', 'precision', 'recall', 'f1_score', 'auc']])

    return best_model, best_config, results_df

# ============================================================================
# 5. TRAIN FINAL MODEL WITH BEST HYPERPARAMETERS
# ============================================================================

def train_final_model(X_train, y_train, X_test, y_test, best_params):
    """
    Train final ANN model with the best hyperparameters
    """
    print("\n" + "="*50)
    print("TRAINING FINAL ANN MODEL")
    print("="*50)

    # Extract best parameters
    best_params_dict = best_params['params']

    # Create final model
    final_model = create_ann_model(
        learning_rate=best_params_dict.get('learning_rate', 0.001),
        hidden_layers=best_params_dict.get('hidden_layers', 2),
        neurons_per_layer=best_params_dict.get('neurons_per_layer', 64),
        dropout_rate=best_params_dict.get('dropout_rate', 0.3),
        activation=best_params_dict.get('activation', 'relu'),
        optimizer=best_params_dict.get('optimizer', 'adam'),
        l1_reg=best_params_dict.get('l1_reg', 0.001),
        l2_reg=best_params_dict.get('l2_reg', 0.001)
    )

    # Display model architecture
    print("\nModel Architecture:")
    print("-" * 30)
    final_model.summary()

    # Define callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=8,
        min_lr=0.00001,
        verbose=1
    )

    # Train the model
    print("\nTraining the model...")
    history = final_model.fit(
        X_train_processed, y_train,
        validation_split=0.2,
        batch_size=best_params_dict.get('batch_size', 32),
        epochs=best_params_dict.get('epochs', 100),
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )

    return final_model, history

# ============================================================================
# 6. MODEL EVALUATION AND VISUALIZATION
# ============================================================================

def evaluate_model(model, X_test, y_test, history=None):
    """
    Evaluate the trained model and create visualizations
    """
    print("\n" + "="*50)
    print("MODEL EVALUATION")
    print("="*50)

    # Make predictions
    y_pred_proba = model.predict(X_test_processed, verbose=0)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred_proba)

    print("\nPerformance Metrics:")
    print("-" * 30)
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print(f"AUC-ROC:   {auc:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)

    # Classification Report
    print("\nClassification Report:")
    print("-" * 30)
    print(classification_report(y_test, y_pred, target_names=['No Churn', 'Churn']))

    # Plotting
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    # 1. Training History - Loss
    if history is not None:
        axes[0, 0].plot(history.history['loss'], label='Training Loss')
        axes[0, 0].plot(history.history['val_loss'], label='Validation Loss')
        axes[0, 0].set_title('Model Loss')
        axes[0, 0].set_xlabel('Epoch')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

    # 2. Training History - Accuracy
    if history is not None:
        axes[0, 1].plot(history.history['accuracy'], label='Training Accuracy')
        axes[0, 1].plot(history.history['val_accuracy'], label='Validation Accuracy')
        axes[0, 1].set_title('Model Accuracy')
        axes[0, 1].set_xlabel('Epoch')
        axes[0, 1].set_ylabel('Accuracy')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

    # 3. Confusion Matrix Heatmap
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Churn', 'Churn'],
                yticklabels=['No Churn', 'Churn'],
                ax=axes[0, 2])
    axes[0, 2].set_title('Confusion Matrix')
    axes[0, 2].set_ylabel('True Label')
    axes[0, 2].set_xlabel('Predicted Label')

    # 4. ROC Curve
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    axes[1, 0].plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {auc:.3f})')
    axes[1, 0].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Random')
    axes[1, 0].set_xlim([0.0, 1.0])
    axes[1, 0].set_ylim([0.0, 1.05])
    axes[1, 0].set_xlabel('False Positive Rate')
    axes[1, 0].set_ylabel('True Positive Rate')
    axes[1, 0].set_title('Receiver Operating Characteristic (ROC) Curve')
    axes[1, 0].legend(loc="lower right")
    axes[1, 0].grid(True, alpha=0.3)

    # 5. Feature Importance (using permutation importance)
    from sklearn.inspection import permutation_importance
    try:
        # Note: This can be computationally expensive
        # Let's use a subset for faster computation
        if X_test_processed.shape[1] > 50:  # If too many features, sample
            sample_indices = np.random.choice(X_test_processed.shape[1], 20, replace=False)
            X_test_sample = X_test_processed[:, sample_indices]
            feature_names_sample = [feature_names[i] for i in sample_indices]
        else:
            X_test_sample = X_test_processed
            feature_names_sample = feature_names[:20] if len(feature_names) > 20 else feature_names

        # Calculate permutation importance
        perm_importance = permutation_importance(
            model, X_test_sample, y_test,
            n_repeats=5,
            random_state=42,
            n_jobs=-1
        )

        # Get top 10 important features
        sorted_idx = perm_importance.importances_mean.argsort()[-10:]
        axes[1, 1].barh(range(len(sorted_idx)),
                       perm_importance.importances_mean[sorted_idx])
        axes[1, 1].set_yticks(range(len(sorted_idx)))
        if len(feature_names_sample) >= 10:
            axes[1, 1].set_yticklabels([feature_names_sample[i] for i in sorted_idx])
        axes[1, 1].set_xlabel('Permutation Importance')
        axes[1, 1].set_title('Top 10 Feature Importances')
    except Exception as e:
        axes[1, 1].text(0.5, 0.5, 'Feature importance\ncalculation failed\n\nError: ' + str(e),
                       ha='center', va='center', transform=axes[1, 1].transAxes)
        axes[1, 1].set_title('Feature Importance')

    # 6. Prediction Distribution
    axes[1, 2].hist(y_pred_proba[y_test == 0], bins=30, alpha=0.5, label='No Churn', color='blue')
    axes[1, 2].hist(y_pred_proba[y_test == 1], bins=30, alpha=0.5, label='Churn', color='red')
    axes[1, 2].axvline(x=0.5, color='green', linestyle='--', label='Decision Threshold')
    axes[1, 2].set_xlabel('Predicted Probability')
    axes[1, 2].set_ylabel('Frequency')
    axes[1, 2].set_title('Prediction Distribution by Class')
    axes[1, 2].legend()

    plt.tight_layout()
    plt.savefig('ann_model_evaluation.png', dpi=300, bbox_inches='tight')
    plt.show()

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc': auc,
        'confusion_matrix': cm,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

# ============================================================================
# 7. MAIN EXECUTION
# ============================================================================

def main():
    """
    Main execution function
    """
    if df.empty:
        print("No data loaded. Please check the dataset path.")
        return

    print("\n" + "="*60)
    print("TELCO CUSTOMER CHURN PREDICTION - ANN IMPLEMENTATION")
    print("="*60)

    # Perform hyperparameter tuning
    best_model, best_config, tuning_results = perform_hyperparameter_tuning(
        X_train_processed, y_train, X_test_processed, y_test
    )

    # Train final model with best parameters
    final_model, history = train_final_model(
        X_train_processed, y_train, X_test_processed, y_test, best_config
    )

    # Evaluate the model
    evaluation_results = evaluate_model(final_model, X_test_processed, y_test, history)

    # Save the model
    final_model.save('telco_churn_ann_model.h5')
    print("\nModel saved as 'telco_churn_ann_model.h5'")

    # Save preprocessing pipeline
    import joblib
    joblib.dump(preprocessor, 'preprocessor.pkl')
    print("Preprocessing pipeline saved as 'preprocessor.pkl'")

    # Print final summary
    print("\n" + "="*60)
    print("FINAL SUMMARY")
    print("="*60)
    print(f"Best Configuration: {best_config['name']}")
    print(f"Best F1-Score: {evaluation_results['f1_score']:.4f}")
    print(f"Best Accuracy: {evaluation_results['accuracy']:.4f}")
    print(f"Best AUC-ROC: {evaluation_results['auc']:.4f}")

    # Display best hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_config['params'].items():
        print(f"  {key}: {value}")

# ============================================================================
# 8. ADDITIONAL UTILITY FUNCTIONS
# ============================================================================

def predict_new_customer(model, preprocessor, customer_data):
    """
    Predict churn probability for a new customer
    """
    # Convert to DataFrame
    customer_df = pd.DataFrame([customer_data])

    # Apply preprocessing
    customer_processed = preprocessor.transform(customer_df)

    # Make prediction
    prediction_proba = model.predict(customer_processed, verbose=0)[0][0]
    prediction = 1 if prediction_proba > 0.5 else 0

    return {
        'churn_probability': float(prediction_proba),
        'prediction': 'Churn' if prediction == 1 else 'No Churn',
        'confidence': float(prediction_proba if prediction == 1 else 1 - prediction_proba)
    }

# Example customer data structure
example_customer = {
    'gender': 'Female',
    'SeniorCitizen': 0,
    'Partner': 'Yes',
    'Dependents': 'No',
    'tenure': 12,
    'PhoneService': 'Yes',
    'MultipleLines': 'No',
    'InternetService': 'DSL',
    'OnlineSecurity': 'No',
    'OnlineBackup': 'Yes',
    'DeviceProtection': 'No',
    'TechSupport': 'No',
    'StreamingTV': 'No',
    'StreamingMovies': 'No',
    'Contract': 'Month-to-month',
    'PaperlessBilling': 'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges': 65.5,
    'TotalCharges': 786.0
}

# ============================================================================
# EXECUTE MAIN FUNCTION
# ============================================================================

if __name__ == "__main__":
    main()

    # Example of using the trained model for prediction
    print("\n" + "="*60)
    print("EXAMPLE PREDICTION FOR NEW CUSTOMER")
    print("="*60)

    # Load the saved model and preprocessor
    try:
        loaded_model = keras.models.load_model('telco_churn_ann_model.h5')
        loaded_preprocessor = joblib.load('preprocessor.pkl')

        prediction_result = predict_new_customer(
            loaded_model, loaded_preprocessor, example_customer
        )

        print(f"Churn Probability: {prediction_result['churn_probability']:.4f}")
        print(f"Prediction: {prediction_result['prediction']}")
        print(f"Confidence: {prediction_result['confidence']:.4f}")
    except:
        print("Model files not found. Run main() first to train and save the model.")

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers.scikit_learn'